In [1]:
import pygame
from pygame.locals import QUIT, KEYDOWN

pygame 2.5.1 (SDL 2.28.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Circuit:
    def __init__(self, x, y, width):
        self.x = x
        self.y = y
        self.width = width
        
        self.matrix = None
        
        self.checkpoints = self.generate_checkpoints()
        
        self.create_matrix()
        
    def generate_checkpoints(self):
        checkpoints = []
        mid_x = (self.x + 2 * self.width) // 2
        mid_y = (self.y + 2 * self.width) // 2

        for i in range(1, self.x + 2 * self.width - 1):
            checkpoints.append((i, 1))
            checkpoints.append((i, self.x + 2 * self.width - 2))
            
        for i in range(1, self.y + 2 * self.width - 1):
            checkpoints.append((self.x + 2 * self.width - 2, i))
            checkpoints.append((1, i))

        return checkpoints  
    
    def create_matrix(self):
        circuit = [['-' for _ in range(self.x + 2 * self.width)] for _ in range(self.y + 2 * self.width)]
    
        for i in range(self.width, self.y + self.width):
            for j in range(self.width, self.x + self.width):
                circuit[i][j] = ' '
                if 1 < i < (self.y):
                    if j == (self.x + 2 * self.width) // 2:
                        circuit[i][j] = '-'
                if i == (self.y + 2 * self.width) // 2:
                    if 1 <= j <= (self.x) // 2:
                        circuit[i][j] = '.'
        for checkpoint in self.checkpoints:
            checkpoint_x, checkpoint_y = checkpoint
            circuit[checkpoint_y][checkpoint_x] = 'c'
        self.matrix = circuit
        
    def update_circuit(self, car, window):
        circuit_with_car = [row.copy() for row in self.matrix]
        car_x, car_y = car.get_position()
        circuit_with_car[car_y][car_x] = 'x'
        
        wall_color = (0, 0, 0)
        track_color = (255, 255, 255)
        car_color = (255, 0, 0)
        checkpoint_color = (0, 255, 0)

        cell_size = 40  # Adjust as needed

        for row_index, row in enumerate(circuit_with_car):
            for col_index, cell in enumerate(row):
                cell_rect = pygame.Rect(
                    col_index * cell_size, row_index * cell_size, cell_size, cell_size
                )
                if cell == '-':
                    pygame.draw.rect(window, wall_color, cell_rect)  # Draw wall
                elif cell == ' ':
                    pygame.draw.rect(window, track_color, cell_rect)  # Draw track
                elif cell == 'x':
                    pygame.draw.rect(window, car_color, cell_rect)  # Draw car
                elif cell == 'c':
                    pygame.draw.rect(window, checkpoint_color, cell_rect)  # Draw checkpoint


    def get_circuit(self):
        print('The Circuit')
        for row in self.matrix:
            print(''.join(row))
            
class Car:
    def __init__(self, circuit):
        self.circuit = circuit
        self.x = circuit.x + 2 * circuit.width - 3  # Initial x-coordinate
        self.y = (circuit.y + 2 * circuit.width) // 2  # Initial y-coordinate
        
        # Lap counter 
        self.lap_count = 0
        self.lap_line_x = (self.circuit.x + 2 * self.circuit.width) // 2  # X-coordinate of the lap line

    def move(self):
        prev_x, prev_y = self.x, self.y

        while True:
            event = keyboard.read_event()
            if event.event_type == keyboard.KEY_DOWN:
                move_direction = event.name.lower()

                if move_direction in ['w', 'a', 's', 'd']:
                    self.update_position(move_direction)
                    break
        
#         self.check_collision(prev_x, prev_y)
#         lap_completed = self.check_lap_completition()
#         return lap_completed

    def update_position(self, direction):
        if direction == 'w':
            self.y -= 1
        elif direction == 's':
            self.y += 1
            self.check_lap_completion()
        elif direction == 'a':
            self.x -= 1
        elif direction == 'd':
            self.x += 1
        self.check_collision(self.x, self.y)
        
        
    def check_lap_completion(self):
        if (self.circuit.matrix[self.y][self.x] == '.'):
            self.lap_count += 1

        
    def check_collision(self, prev_x, prev_y):
        if (self.circuit.matrix[self.y][self.x] == '-'):
            self.x = circuit.x + 2 * circuit.width - 2
            self.y = (circuit.y + 2 * circuit.width) // 2

    def get_position(self):
        return self.x, self.y

In [3]:
class CarSimulation:
    def __init__(self, circuit, car):
        self.circuit = circuit
        self.car = car

        pygame.init()
        self.window_width = self.circuit.x / 2 * 100
        self.window_height = self.circuit.y / 2 * 100
        self.window = pygame.display.set_mode((self.window_width, self.window_height))
        pygame.display.set_caption("Car Simulation")

        self.font = pygame.font.Font(None, 36)
        self.clock = pygame.time.Clock()

        self.show_start_screen()

    def show_start_screen(self):
        self.window.fill((0, 0, 0))
        start_text = self.font.render("Press space bar to start the game", True, (255, 255, 255))
        start_rect = start_text.get_rect(center=(self.window_width // 2, self.window_height // 2))
        self.window.blit(start_text, start_rect)
        pygame.display.flip()

        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                    waiting = False

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == QUIT:
                    running = False
                elif event.type == KEYDOWN:  # Check for keydown events
                    move_directions = {
                        pygame.K_w: 'w',
                        pygame.K_s: 's',
                        pygame.K_a: 'a',
                        pygame.K_d: 'd'
                    }
                    if event.key in move_directions:
                        self.car.update_position(move_directions[event.key])

            self.window.fill((0, 0, 0))  # Clear the screen
            self.circuit.update_circuit(self.car, self.window)
            self.display_lap_counter()  # Display lap counter on the screen
            pygame.display.update()

            self.clock.tick(60)  # Limit to 60 FPS

        pygame.quit()

    def display_lap_counter(self):
        lap_text = self.font.render(f"Lap: {self.car.lap_count}", True, (255, 255, 255))
        lap_rect = lap_text.get_rect(topleft=(10, 10))
        self.window.blit(lap_text, lap_rect)

# Example usage
x = 9
y = 9
width = 1
circuit = Circuit(x, y, width)
car = Car(circuit)
simulation = CarSimulation(circuit, car)
simulation.run()
